In [2]:
import os
import netCDF4
import geopandas as gpd
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
from pathlib import Path

In [ ]:
# Get point grid (pg) identifiers from https://dgg.geo.tuwien.ac.at/
pg_finland = "/Users/casado/projects/CASSINI/Download_SWI/pointlist_Finland_copernicus_swi_ts.csv"

In [49]:
import cdsapi

c = cdsapi.Client()

c.retrieve(
    'satellite-soil-moisture',
    {
        'format': 'zip',
        'variable': 'surface_soil_moisture',
        'time_aggregation': 'day_average',
        'year': [
            '2017', '2018', '2019',
            '2020',
        ],
        'month': [
            '05', '06', '07',
            '08',
        ],
        'day': [
            '01', '02', '03',
            '04', '05', '06',
            '07', '08', '09',
            '10', '11', '12',
            '13', '14', '15',
            '16', '17', '18',
            '19', '20', '21',
            '22', '23', '24',
            '25', '26', '27',
            '28', '29', '30',
            '31',
        ],
        'type_of_record': 'cdr',
        'version': 'v202012.0.0',
        'type_of_sensor': 'active',
    },
    'download.zip')

2022-11-05 20:32:05,544 INFO Welcome to the CDS
2022-11-05 20:32:05,545 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/satellite-soil-moisture
2022-11-05 20:32:05,617 INFO Request is queued
2022-11-05 20:32:06,673 INFO Request is running
2022-11-05 20:38:24,241 INFO Request is completed
2022-11-05 20:38:24,241 INFO Downloading https://download-0014-clone.copernicus-climate.eu/cache-compute-0014/cache/data0/dataset-satellite-soil-moisture-f8a37485-3775-4c81-9601-eec205e7c2d9.zip to download.zip (817.4M)
2022-11-05 20:57:49,399 INFO Download rate 718.4K/s


Result(content_length=857119050,content_type=application/zip,location=https://download-0014-clone.copernicus-climate.eu/cache-compute-0014/cache/data0/dataset-satellite-soil-moisture-f8a37485-3775-4c81-9601-eec205e7c2d9.zip)

## CDSAPI downloaded empty files 
Next we read the files downloaded with `fetch-copernicus-swi-data.sh` 

In [3]:
path="/Users/casado/projects/CASSINI/FinAware/src/data/land.copernicus.vgt.vito.be/PDF/datapool/Vegetation/Soil_Water_Index/Daily_SWI_1km_Europe_V1/2015/07/01/SWI1km_201507011200_CEURO_SCATSAR_V1.0.1/c_gls_SWI1km_201507011200_CEURO_SCATSAR_V1.0.1.nc"
nc = netCDF4.Dataset(path)
nc.variables['SWI_015'][:]

masked_array(
  data=[[[--, --, --, ..., --, --, --],
         [--, --, --, ..., --, --, --],
         [--, --, --, ..., --, --, --],
         ...,
         [--, --, --, ..., --, --, --],
         [--, --, --, ..., --, --, --],
         [--, --, --, ..., --, --, --]]],
  mask=[[[ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True],
         ...,
         [ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True]]],
  fill_value=255,
  dtype=float32)

In [4]:
## Read from the 1km-res dataset (one file as example)
path="/Users/casado/projects/CASSINI/FinAware/src/data/land.copernicus.vgt.vito.be/PDF/datapool/Vegetation/Soil_Water_Index/Daily_SWI_1km_Europe_V1/2015/07/01/SWI1km_201507011200_CEURO_SCATSAR_V1.0.1/c_gls_SWI1km_201507011200_CEURO_SCATSAR_V1.0.1.nc"
ds_disk = xr.open_dataset(path)
ds_disk

<xarray.Dataset>
Dimensions:    (lat: 4144, lon: 6832, time: 1)
Coordinates:
  * lat        (lat) float64 72.0 71.99 71.98 71.97 ... 35.03 35.02 35.01 35.0
  * lon        (lon) float64 -11.0 -10.99 -10.98 -10.97 ... 49.98 49.99 50.0
  * time       (time) datetime64[ns] 2015-07-01T12:00:00
Data variables: (12/18)
    crs        |S1 ...
    SSF        (time, lat, lon) float32 ...
    SWI_002    (time, lat, lon) float32 ...
    QFLAG_002  (time, lat, lon) float32 ...
    SWI_005    (time, lat, lon) float32 ...
    QFLAG_005  (time, lat, lon) float32 ...
    ...         ...
    SWI_040    (time, lat, lon) float32 ...
    QFLAG_040  (time, lat, lon) float32 ...
    SWI_060    (time, lat, lon) float32 ...
    QFLAG_060  (time, lat, lon) float32 ...
    SWI_100    (time, lat, lon) float32 ...
    QFLAG_100  (time, lat, lon) float32 ...
Attributes: (12/26)
    Conventions:               CF-1.6
    archive_facility:          VITO
    copyright:                 Copernicus Service information 2019
    geospatial_lat_max:        72.0
    geospatial_lat_min:        35.0
    geospatial_lon_max:        50.0
    ...                        ...
    region_name:               CEURO
    sensor:                    ASCAT, CSAR
    source:                    Derived from EO radar observations
    time_coverage_end:         2015-07-01T12:00:00Z
    time_coverage_start:       2015-06-30T12:00:01Z
    title:                     Daily Soil Water Index 1km: CEURO 2015-07-01T1...

In [5]:
SWI15 = ds_disk.SWI_015
SWI15.attrs = ds_disk.attrs
SWI15

<xarray.DataArray 'SWI_015' (time: 1, lat: 4144, lon: 6832)>
[28311808 values with dtype=float32]
Coordinates:
  * lat      (lat) float64 72.0 71.99 71.98 71.97 ... 35.03 35.02 35.01 35.0
  * lon      (lon) float64 -11.0 -10.99 -10.98 -10.97 ... 49.97 49.98 49.99 50.0
  * time     (time) datetime64[ns] 2015-07-01T12:00:00
Attributes:
    Conventions:               CF-1.6
    archive_facility:          VITO
    copyright:                 Copernicus Service information 2019
    geospatial_lat_max:        72.0
    geospatial_lat_min:        35.0
    geospatial_lon_max:        50.0
    geospatial_lon_min:        -11.0
    history:                   2019-03-14 -- Reprocessing Production
    identifier:                urn:cgls:global:swi_v1_1km:SWI1km_201507011200...
    institution:               TU Wien
    long_name:                 Soil Water Index
    model_parameter_baseline:  2015 - 2018
    orbit_type:                LEO
    parent_identifier:         urn:cgls:global:swi_v1_1km
    pixel_size:                0.0089285714 degrees
    platform:                  METOP A+B, Sentinel-1 A+B
    processing_level:          L3
    processing_mode:           Reprocessing
    product_version:           V1.0.1
    references:                https://land.copernicus.eu/global/products/swi
    region_name:               CEURO
    sensor:                    ASCAT, CSAR
    source:                    Derived from EO radar observations
    time_coverage_end:         2015-07-01T12:00:00Z
    time_coverage_start:       2015-06-30T12:00:01Z
    title:                     Daily Soil Water Index 1km: CEURO 2015-07-01T1...

In [1]:
gpd.read_file('starch_fields_projected.gpkg')

NameError: name 'gpd' is not defined

In [49]:
with xr.open_dataset(path) as ds:
    print(ds.keys())

KeysView(<xarray.Dataset>
Dimensions:    (lat: 4144, lon: 6832, time: 1)
Coordinates:
  * lat        (lat) float64 72.0 71.99 71.98 71.97 ... 35.03 35.02 35.01 35.0
  * lon        (lon) float64 -11.0 -10.99 -10.98 -10.97 ... 49.98 49.99 50.0
  * time       (time) datetime64[ns] 2015-07-01T12:00:00
Data variables: (12/18)
    crs        |S1 ...
    SSF        (time, lat, lon) float32 ...
    SWI_002    (time, lat, lon) float32 ...
    QFLAG_002  (time, lat, lon) float32 ...
    SWI_005    (time, lat, lon) float32 ...
    QFLAG_005  (time, lat, lon) float32 ...
    ...         ...
    SWI_040    (time, lat, lon) float32 ...
    QFLAG_040  (time, lat, lon) float32 ...
    SWI_060    (time, lat, lon) float32 ...
    QFLAG_060  (time, lat, lon) float32 ...
    SWI_100    (time, lat, lon) float32 ...
    QFLAG_100  (time, lat, lon) float32 ...
Attributes: (12/26)
    Conventions:               CF-1.6
    archive_facility:          VITO
    copyright:                 Copernicus Service inform

In [42]:
BASE_PATH='land.copernicus.vgt.vito.be/'
FILE_SUFFIX='*.nc'
file_list = list(Path(BASE_PATH).rglob(FILE_SUFFIX))



oil_Water_Index/Daily_SWI_1km_Europe_V1/2015/02/15/SWI1km_201502151200_CEURO_SCATSAR_V1.0.1/c_gls_SWI1km_201502151200_CEURO_SCATSAR_V1.0.1.nc'), PosixPath('land.copernicus.vgt.vito.be/PDF/datapool/Vegetation/Soil_Water_Index/Daily_SWI_1km_Europe_V1/2015/02/12/SWI1km_201502121200_CEURO_SCATSAR_V1.0.1/c_gls_SWI1km_201502121200_CEURO_SCATSAR_V1.0.1.nc'), PosixPath('land.copernicus.vgt.vito.be/PDF/datapool/Vegetation/Soil_Water_Index/Daily_SWI_1km_Europe_V1/2015/02/13/SWI1km_201502131200_CEURO_SCATSAR_V1.0.1/c_gls_SWI1km_201502131200_CEURO_SCATSAR_V1.0.1.nc'), PosixPath('land.copernicus.vgt.vito.be/PDF/datapool/Vegetation/Soil_Water_Index/Daily_SWI_1km_Europe_V1/2015/02/14/SWI1km_201502141200_CEURO_SCATSAR_V1.0.1/c_gls_SWI1km_201502141200_CEURO_SCATSAR_V1.0.1.nc'), PosixPath('land.copernicus.vgt.vito.be/PDF/datapool/Vegetation/Soil_Water_Index/Daily_SWI_1km_Europe_V1/2015/02/22/SWI1km_201502221200_CEURO_SCATSAR_V1.0.1/c_gls_SWI1km_201502221200_CEURO_SCATSAR_V1.0.1.nc'), PosixPath('land.cop